# Gradient Descent Nedir?

Hatırlayalım, __least squares method__ ile çalışırken elimizdeki veriye göre hata fonksiyonumuzu minimize eden parametreleri bulmaya çalışıyorduk.

Sonrasında birkaç adım sonucunda bize bu optimal parametreleri veren bir denklem elde ettik:

$ w^* = (X^TX)^{-1}X^TY$

Bu çözüm bize direkt olarak optimal parametreleri verdiği için __closed form__ olarak adlandırılır demiştik, fakat bu çözüm her zaman mümkün olmayabilir.
Buna güzel bir örnek elimizde çok fazla veri olduğu durum, $ X \in \mathbb{R}^{N \times D}$ olduğu durumda, çok fazla veri olması, $N$'in çok büyük olması anlamına geliyor.
Bugün işe yarar bir veri setinden bahsettiğimizde $N$'in milyonlar hatta milyarlar seviyesinde büyük olduğunu düşünebiliriz, burada bahsetmediğimiz başka bazı ince detayları da
düşündüğümüzde bu çözümün uygulanabilir olmadığını görebiliriz.

O zaman bir alternatife ihtiyacımız var, öyle bir alternatif ki bizi sadece maaliyetli matris çarpımlarından kurtarmayacak, bir adım ötesinde hem hafızaya sığmayacak
büyüklükte veri kümelerini kullanmak için bize bir yol sunacak hem de direkt olarak optimal parametreleri bulamayacağımız daha komplike hata fonksiyonlarını minimize etmekte
hiçbir sorun yaşamayacak.

Bu alternatif __Gradient Descent__. Şimdilik onu sadece bizi matris tersi gibi işlemlerden kurtarıyor gibi düşünelim.

Gradient Descent, bir fonksiyonun minimum noktasına ulaşmak için kullanılan bir optimizasyon algoritmasıdır. Bu algoritma bizi direkt olarak minimum noktaya götürmez,
ama aşama aşama bizi minimum noktasına yaklaştırır. Burada kendinize sorabileceğiniz güzel bir soru, bu algoritmanın çalışması için belli şartlar var mı? Bunu toplantılarda tartışabiliriz.

Bu __notebook__'da Gradient Descent'in niye çalıştığı ile ilgili çok kısa bir matematiksel tanımın ardından bolca sezgisel örnek inceleyeceğiz.

## Gradient Descent'in Matematiksel Tanımı

Öncelikle Gradient yani gradyan nedir bunun üstünde durmamız lazım, aslında burada bahsedilecek pek bir şey yok, bir fonksiyonun belli bir noktadaki gradyanı bize o fonksiyonun
en hızlı arttığı yönü verir. Bir dağın yüzeyini bir fonksiyon olarak düşünürseniz ve kendinizi de bu dağın üstünde hayal ederseniz, dağı modelleyen fonksiyonun sizin bulunduğunuz
noktadaki gradyanı size 2 boyutlu bir vektör verir, bu vektör sizin hangi yönde 1 birim ilerlediğinizde en çok yükseleceğinizi gösteren vektördür. Kısaca en hızlı artış yönü diyebiliriz.

Büyük ihtimalle nereye varıyor olduğumu anladınız, eğer gradyan yönünde gittiğimde yüksekliğim en hızlı artıyorsa, gradyanın tersi yönde gittiğimde de bu yüksekliğimin en hızlı azalacağı
anlamına gelir. Bizim hata fonksiyonu dediğimiz fonksiyonu da bir dağ olarak düşünürsek, en dibe ulaşmak, hatanın en az olduğu optimumu noktaya ulaşmak demek, o yüzden bu yönde ilerlersek
ulaşmak istediğimiz minimum noktasına ulaşabiliriz, ama tabii ki burada değerlendirmemiz gereken bazı şeyler var.

Şimdi gelin birkaç örnek ile bu konuyu daha iyi anlamaya çalışalım.

## 1 Boyutlu Örnek

İnternetteki kaynaklarda Gradient Descent ile ilgili çok güzel görselleştirmeler mevcut, fakat benim bunlarla ilgili en büyük sorunum her zaman şu oldu, 3 boyutlu örnekler bize biraz
yanlış bir algı veriyor olabilir. Az önce bahsettiğim örneği düşünün, bir dağdan bahsettim, dağ 3 boyutlu bir obje ve gerçekten de baktığınızda ortada 3 boyutlu bir anlatım var, ama şu
çok önemli. Dağ örneğinde yükseklik bizim için fonksiyonun sonucuydu, dağda yürüme örneği bizim yüksekliğimizi de bilinçli olarak değiştirdiğimiz yanılgısını veriyor ama bu örnekte gerçekte
olan şey biz sadece 2 boyutlu hareket ediyoruz, öne arkaya, sağa ve sola daha sonra 2 boyutlu konumumuza göre fonksiyonumuz bize bir yükseklik tayin ediyor. Hatta daha anlamlı olması için şöyle diyelim
biz sadece enlem ve boylamımızı değiştiriyoruz, yani dağ fonksiyonu şu şekilde gözükecek:

$ f(enlem, boylam) = yükseklik $

Bunun başta garip geldiğinin farkındayım, bana da öyle olmuştu. Garip olmasının sebebi gerçek 3 boyutlu bir dünyada biz sadece enlem ve boylamımızı değiştirmiyoruz aynı zamanda kendimizi yukarı taşıyoruz bu yüzden
3 boyutlu bir hareket yapıyor oluyoruz. Yani bize Gradient Descent'i aslında oldukça iyi anlatan bu örnekte, bu ince ayrımın farkında olmamız gerekiyor. Şimdi 1 boyutlu örnekte demek istediğimi çok daha iyi anlayacaksınız.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

In [ ]:
# Basit bir x^2 foknsiyonunu düşünelim.

x = np.linspace(-3, 3, 100)
y = x ** 2

plt.plot(x, y)
plt.show()

Burada herhangi bir noktada gradyan aldığımızda, bu gradyanın şöyle gözüktüğünü düşünmeye meyilliyiz:

In [ ]:
# Basit bir x^2 foknsiyonunu düşünelim.

x = np.linspace(-3, 3, 100)
y = x ** 2

x0 = 1
y0 = x0 ** 2

x_pseudo_grad = (0.5, x0 + y0)

plt.plot(x, y)
plt.scatter(x0, y0, c="red")
plt.arrow(x0, y0, x_pseudo_grad[0], x_pseudo_grad[1], head_width=0.2, head_length=0.2, color="red")
plt.show()

Ama aslında az önce açıkladığımız sebeplerden burada $f(x) = x^2$ fonksiyonunun x'e göre gradyanını aldığımız için aslında $y$ değeri üzerinde hiçbir kontrolümüz yok, biz sadece $x$ değerini değiştiriyoruz.

O halde gerçek gradyan şöyle gözükecek:

In [ ]:
# Basit bir x^2 foknsiyonunu düşünelim.

x = np.linspace(-3, 3, 100)
y = x ** 2

x0 = 1
y0 = x0 ** 2

x_grad = 2 * x0

plt.plot(x, y)
plt.scatter(x0, y0, c="red")
plt.arrow(x0, y0, x_grad, 0, head_width=0.2, head_length=0.2, color="red")
plt.show()

Tam bu vektör kadar hareket ettiğimizde ise yeni noktamız şöyle olacak:

In [ ]:
# Basit bir x^2 foknsiyonunu düşünelim.

x = np.linspace(-3, 3, 100)
y = x ** 2

x0 = 1
y0 = x0 ** 2

x_grad = 2 * x0

x1 = x0 + x_grad

plt.plot(x, y)
plt.scatter(x0, y0, c="red")
plt.arrow(x0, y0, x_grad, 0, head_width=0.2, head_length=0.2, color="red")
plt.scatter(x1, x1 ** 2, c="green")
plt.plot([x1, x1], [0, x1 ** 2], "--", c="green")
plt.show()

# 2 Boyutlu Örnek

Bu konsepti daha iyi oturtmak için şimdi bir de 2 boyutlu bir örneğe bakalım:

In [ ]:
x = np.linspace(-3, 3, 100)
y = np.linspace(-3, 3, 100)

x, y = np.meshgrid(x, y)

z = x ** 2 + y ** 2

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(x, y, z)
plt.show()



In [ ]:
x = np.linspace(-3, 3, 100)
y = np.linspace(-3, 3, 100)

x, y = np.meshgrid(x, y)

z = x ** 2 + y ** 2

x0, y0 = -2, 2

fig = plt.figure()
ax = plt.axes(projection='3d')

ax.plot_surface(x, y, z, alpha=0.8)
ax.plot([x0 + 0.2], [y0 + 0.2], [x0 ** 2 + y0 ** 2 + 0.2], "ro", markersize=10, alpha=0.8, zorder=10)
plt.show()



Bu durumda gradyan da şöyle gözükecek:

In [ ]:
x = np.linspace(-3, 3, 100)
y = np.linspace(-3, 3, 100)

x, y = np.meshgrid(x, y)

z = x ** 2 + y ** 2

x0, y0 = -2, 2

grad = np.array([x0, y0, 0])

fig = plt.figure()
ax = plt.axes(projection='3d')

ax.plot_surface(x, y, z, alpha=0.8)
ax.plot([x0 + 0.2], [y0 + 0.2], [x0 ** 2 + y0 ** 2 + 0.2], "ro", markersize=10, alpha=0.8, zorder=10)

ax.quiver(x0 + 0.2, y0 + 0.2, x0 ** 2 + y0 ** 2 + 0.2, grad[0], grad[1], 0, color="red")
plt.show()



Görünüşü sizi yanılmasın, birazcık döndürürsek şöyle gözükecek:

In [ ]:
x = np.linspace(-3, 3, 100)
y = np.linspace(-3, 3, 100)

x, y = np.meshgrid(x, y)

z = x ** 2 + y ** 2

x0, y0 = -2, 2

grad = np.array([x0, y0, 0])

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.view_init(40, 60)

ax.plot_surface(x, y, z, alpha=0.8)
ax.plot([x0 + 0.2], [y0 + 0.2], [x0 ** 2 + y0 ** 2 + 0.2], "ro", markersize=10, alpha=0.8, zorder=10)
ax.quiver(x0 + 0.2, y0 + 0.2, x0 ** 2 + y0 ** 2 + 0.2, grad[0], grad[1], 0, color="red")
plt.show()

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.view_init(0, 30)

ax.plot_surface(x, y, z, alpha=0.8)
ax.plot([x0 + 0.2], [y0 + 0.2], [x0 ** 2 + y0 ** 2 + 0.2], "ro", markersize=10, alpha=0.8, zorder=10)
# Add 3D arrow
ax.quiver(x0 + 0.2, y0 + 0.2, x0 ** 2 + y0 ** 2 + 0.2, grad[0], grad[1], 0, color="red")
plt.show()



Aynı şeyi 2D bir plot kullanarak da şu şekilde gösterebiliriz:

In [ ]:
x = np.linspace(-3, 3, 100)
y = np.linspace(-3, 3, 100)

x, y = np.meshgrid(x, y)

z = x ** 2 + y ** 2

x0, y0 = -2, 2

grad = np.array([2 * x0, 2 * y0, 0])

fig = plt.figure()
plt.contour(x, y, z, 50, cmap="viridis")
plt.plot([x0], [y0], "ro", markersize=10, alpha=0.8, zorder=10)
plt.quiver(x0, y0, grad[0], grad[1], color="red")
plt.show()



Böylece optimize ettiğimiz değeri sadece renk ve kontürler şeklinde görürken plotun kendisinde sadece hareket kabiliyetimiz olan yönleri görüyoruz ve aslında bu
aklımıza daha çok yatacak bir görselleştirme oluyor.

# Gradient Descent'i Nasıl Uyguylayacağız?

Bu noktaya kadar aslına sadece gradyandan bahsettik, ama bu bile bize Gradient Descent algoritmasını nasıl kullanacağımız ile ilgili neredeyse
tüm bilgiyi sağladı.

Şimdi Gradient Descent'i nasıl yapabilirdik düşünelim, madem gradyan bize en hızlı artış yönünü ve gradyanın negatifi en hızlı düşüş yönünü gösteriyor,
o zaman iteratif olarak negatif gradyan yönünde gidersek minimuma ulaşabiliriz. Bunu görselleştirelim (Bu sefer bir tık daha karışık bir fonksiyon kullanacağız):

In [ ]:
x = np.linspace(-3, 3, 100)
y = np.linspace(-3, 3, 100)

x, y = np.meshgrid(x, y)

z = 2 * x ** 4 + y ** 2 - 4 * x

x0, y0 = -1.4, 2

for idx in range(3):

    grad = np.array([
        8 * x0 ** 3 - 4,
        2 * y0,
        0
    ])

    fig = plt.figure()
    plt.contour(x, y, z, 50, cmap="viridis")
    plt.plot([x0], [y0], "ro", markersize=10, alpha=0.8, zorder=10)
    plt.quiver(x0, y0, grad[0], grad[1], color="red")
    plt.title(f"Iteration {idx}")
    plt.show()

    # Noktaları gradyanın tersi yönünde ilerletelim.
    x0 = x0 - grad[0]
    y0 = y0 - grad[1]
    
    print(f"X0: {x0}, Y0: {y0}")


Bilgisayarınızın ayarları ile oynamayın, böyle olması çok normaldi. Gradyanı direkt olarak kullanmak ile ilgili ciddi bir problem var, çünkü gradyan bize sadece belli bir yön sağlamıyor, aynı zamanda uzunluğu bize artığın miktarını veriyor,
yani çok dik bir noktadaysak vektörümüzün uzunluğu çok büyük olacak ve bizi istediğimizden çok çok daha fazla hareket ettirecek, bu yüzden __learning rate__ dediğimiz kavramı kullanıyoruz, bu tamamen gradyan vektörümüzün bu kadar büyük adımlar
atmasını istemediğimiz ve adım adım ilerlemek istediğimiz için, gelin bir de bu şekilde deneyelim:

In [ ]:
x = np.linspace(-3, 3, 100)
y = np.linspace(-3, 3, 100)

x, y = np.meshgrid(x, y)

z = 2 * x ** 4 + y ** 2 - 4 * x

x0, y0 = -1.4, 2

for idx in range(10):

    grad = np.array([
        8 * x0 ** 3 - 4,
        2 * y0,
        0
    ])

    fig = plt.figure()
    plt.contour(x, y, z, 50, cmap="viridis")
    plt.plot([x0], [y0], "ro", markersize=10, alpha=0.8, zorder=10)
    plt.quiver(x0, y0, grad[0], grad[1], color="red")
    plt.title(f"Iteration {idx}")
    plt.show()

    # Noktaları gradyanın tersi yönünde ilerletelim.
    x0 = x0 - grad[0] * 0.1
    y0 = y0 - grad[1] * 0.1
    
    print(f"X0: {x0}, Y0: {y0}")


Bu şekilde yaptığımızda gördüğünüz gibi çok daha iyi çalışıyor, burada __learning rate__'i seçmek çoğunlukla deneme yanılmaya kalıyor, genelde belli problemler için ve belli modeller için
bu değeri ne seçeceğimiz aşağı yukarı bellidir, ama yine de kesin bir tespit metodundan bahsetmek çok da mümkün değil.

Tabii bu doğru adımlar atma problemi sadece learning rate'i ayarlamak ile çözülmüyor, bu problemi çok daha karmaşık problemlerde çok daha iyi çözmek için gradient descent'in farklı alternatifleri
mevcut, ama bu alternatiflere burada girmeyeceğiz, notebook'un sonunda bunlara bakmanız için sizlere birkaç link vereceğim.

Şimdi asıl olan olaya gelelim ve görselleştirmeleri bırakalım. Gradient Descent'i matematiksel olarak ifade etmemiz gerekirse, her bir iterasyonu şu şekilde tanımlayabiliriz:

$ x_{n+1} = x_n - \alpha \nabla f(x_n) $

Burada $x_n$ bizim bulunduğumuz nokta, $x_{n+1}$ ise bir sonraki iterasyonda bulunacağımız nokta, $\alpha$ ise learning rate'imiz, $\nabla f(x_n)$ ise gradyanımız.

Şimdi geçen sefer kullandığımız __closed form__ least squares method fonksiyonlarını geri getirelim ve bunlardan aldığımız sonuç ile Gradient Descent'in sonucunu karşılaştıralım:

In [ ]:
def E(w: np.ndarray, X: np.ndarray, Y: np.ndarray) -> float:
    """ Bu fonksiyon parametre olarak aldığı w ağırlık vektörü için
    ortalama hata kareler toplamını (mean squared error) hesaplayıp geri döndürsün.

    Önemli not 1: For veya while döngüsü kullanmadan, sadece matris işlemleri ile bu fonksiyonu yazın.

    Args:
        w (numpy.ndarray): Ağırlık vektörü. Boyutu D x 1.

    Returns:
        float: Hata değeri.
    """

    err = 0.0 # Hesaplandıktan sonra döndürülecek hata değeri

    # TODO: Hata değerini hesaplayın ve err değişkenine atayın.

    diff = Y - X @ w
    err = np.sum(diff ** 2)

    return err


def find_best_w(X: np.ndarray, Y: np.ndarray) -> np.ndarray:
    """ Bu fonksiyon parametre olarak aldığı X ve Y matrislerini kullanarak
    en iyi w değerini bulup geri döndürür.

    Args:
        X (numpy.ndarray): X matrisi. Boyutu N x D.
        Y (numpy.ndarray): Y matrisi. Boyutu N x 1.

    Returns:
        numpy.ndarray: En iyi w değeri. Boyutu D x 1.
    """

    w = np.zeros((X.shape[1], 1)) # En başta w değerimizi 0'lar ile başlatalım.

    # TODO: w değerini bulun ve w değişkenine atayın.

    w = np.linalg.inv(X.T @ X) @ X.T @ Y

    return w

In [ ]:
# Burada tamamen yapay çok boyutlu bir veri seti oluşturuyoruz.

# Tamamen rastgele bir X matrisi oluşturalım.
X = np.random.randn(10000, 10)

# W değerlerimizi de rastgele oluşturalım.
W = np.random.randn(10, 1)

# Y değerlerimizi X matrisi ile W matrisinin çarpımı olarak oluşturalım.
Y = X @ W

# Ufak bir parça gürültü ekleyelim.
Y += np.random.randn(10000, 1) * 0.1

# Burada sadece X ve Y matrislerini kullanarak az önce yarattığımız W değerini bulmaya çalışacağız.


In [ ]:
# Closed form çözüm

w_best = find_best_w(X, Y)

print(f"En iyi w değeri: {w_best}")

print(f"Gerçek w değeri: {W}")

hata = E(w_best, X, Y)

print(f"En iyi w değeri ile elde edilen ortalama hata kareler toplamı: {hata}")

Gördüğünüz gibi oldukça iyi çalışıyor, peki ya Gradient Descent?

In [ ]:
def find_best_w_gradient_descent(X, Y, learning_rate=0.0001, num_iterations=10000):
    """ Bu fonksiyon parametre olarak aldığı X ve Y matrislerini kullanarak
    en iyi w değerini bulup geri döndürür.

    Args:
        X (numpy.ndarray): X matrisi. Boyutu N x D.
        Y (numpy.ndarray): Y matrisi. Boyutu N x 1.
        learning_rate (float, optional): Öğrenme oranı. Defaults to 0.0001.
        num_iterations (int, optional): İterasyon sayısı. Defaults to 10000.

    Returns:
        numpy.ndarray: En iyi w değeri. Boyutu D x 1.
    """

    N, D = X.shape # N ve D değerlerini alalım

    # En başta w değerimizi rastgele ufak değerler ile başlatalım.
    w = np.random.randn(D, 1) * 0.00001

    # Şimdi de istediğimiz sayıda iterasyon yapalım
    for i in range(num_iterations):

        # Hatanın w'ya göre gradyanını hesaplayalım (Önceki örnekte bunu kağıt üstünde bulduktan sonra 0'a eşitliyorduk)
        # Bu sefer diretk olarak gradyanın kendisini hesaplayacağız
        grad = ...

        # Burada ufak bir değişikliğe gitmemiz gerekiyor, önceden hatayı direkt olarak kullanıyorduk, toplam hata örnek
        # sayımız arttıkça büyüyeceği için hatayı örnek sayısına bölerek ortalama bir hata bulduğumuzu düşünelim
        # Bu durumda basit bir sabit ile çarpma işlemi yaptığımız için hem sonuç olarak çıkan gradyanı hem de hata
        # değerini direkt olarak N'e bölebiliriz

        grad = grad / N

        # W değerini güncelleyelim (Gradient Descent)
        w = ...

        # Her 1000 iterasyonda bir hatayı ekrana yazdıralım
        if (i+1) % 5000 == 0:
            hata = E(w, X, Y) / N
            print(f"{i+1}. iterasyon, ortalama hata kareler toplamı: {hata}")

    return w


In [ ]:
# Gradient descent ile w değerini bulalım

# 1000 ile 10000 iterasyon arasında bir değer seçebilirsiniz.i
# Learning rate 1.e-3 ile 1.e-5 arasında iyi çalışacaktır.
w_best = ...

print(f"En iyi w değeri: {w_best}")

print(f"Gerçek w değeri: {W}")

hata = E(w_best, X, Y)

print(f"En iyi w değeri ile elde edilen ortalama hata kareler toplamı: {hata}")

Eğer 100'e yakın bir hata ve orijinal $w$'ya yakın bir $w$ değeri elde ettiyseniz, tebrikler, Gradient Descent'i başarıyla uyguladınız.